In [1]:
%%capture
!pip install -U gdown

In [2]:
!pip install datasets

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 212 kB 46.2 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 1.1 MB 48.8 MB/s 
     |████████████████████████████████| 136 kB 50.1 MB/s 
     |████████████████████████████████| 127 kB 47.2 MB/s 
     |████████████████████████████████| 271 kB 44.5 MB/s 
     |████████████████████████████████| 144 kB 48.7 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
# ! pip install transformers datasets
! pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-2benwdi7
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-2benwdi7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 36.3 MB/s 
     |████████████████████████████████| 596 kB 47.0 MB/s 
  Created wheel for transformers: filename=transformers-4.19.0.dev0-py3-none-any.whl size=4040857 sha256=293db174cb46bad896713220c3fbb86dbc865ad7527c7ced5127aeca3104dc01
  Stored in directory: /tmp/pip-ephem-wheel-cache-p94i531j/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!gdown 1C_K5ISCSmSQVU2uFDBhS5LomQqtCtjJu

Downloading...
From: https://drive.google.com/uc?id=1C_K5ISCSmSQVU2uFDBhS5LomQqtCtjJu
To: /content/Tweets.csv
100% 7.54M/7.54M [00:00<00:00, 77.9MB/s]


In [5]:
import pandas as pd
import numpy as np
import datasets

In [6]:
tweets = pd.read_csv('/content/Tweets.csv')

In [7]:
tweets.columns

Index(['ScreenName', 'Party', 'Text', 'ViralityScore_NormalizedByUserMedian',
       'ViralityScoreSimple_NormalizedByUserMedian',
       'ViralityScore_NormalizedByTotalMedian',
       'ViralityScoreSimple_NormalizedByTotalMedian'],
      dtype='object')

In [8]:
np.percentile(tweets.ViralityScore_NormalizedByTotalMedian, 66)

1.004253540564894

In [9]:
np.percentile(tweets.ViralityScore_NormalizedByTotalMedian, 33)

-1.1481126437509244

In [10]:
rep_tweets = tweets[tweets.Party == 'republican']
dem_tweets = tweets[tweets.Party == 'democrat']

rep_viral_tweets = rep_tweets[rep_tweets.ViralityScoreSimple_NormalizedByUserMedian > 1.004253540564894]
rep_nonviral_tweets = rep_tweets[rep_tweets.ViralityScoreSimple_NormalizedByUserMedian < -1.1481126437509244]

dem_viral_tweets = dem_tweets[dem_tweets.ViralityScoreSimple_NormalizedByUserMedian > 1.004253540564894]
dem_nonviral_tweets = dem_tweets[dem_tweets.ViralityScoreSimple_NormalizedByUserMedian < -1.1481126437509244]


rep_viral_tweets_impt = rep_viral_tweets.loc[      : ,['Text', 'ViralityScoreSimple_NormalizedByUserMedian']]
dem_viral_tweets_impt = dem_viral_tweets.loc[      : ,['Text', 'ViralityScoreSimple_NormalizedByUserMedian']]
rep_nonviral_tweets_impt = rep_nonviral_tweets.loc[: ,['Text', 'ViralityScoreSimple_NormalizedByUserMedian']]
dem_nonviral_tweets_impt = dem_nonviral_tweets.loc[: ,['Text', 'ViralityScoreSimple_NormalizedByUserMedian']]

In [11]:
from sklearn.model_selection import train_test_split

rep_viral_full_train, rep_viral_df_test = train_test_split(rep_viral_tweets_impt, test_size=0.2)
rep_viral_df_train, rep_viral_df_valid = train_test_split(rep_viral_full_train, test_size = 0.2)

dem_viral_full_train, dem_viral_df_test = train_test_split(dem_viral_tweets_impt, test_size=0.2)
dem_viral_df_train, dem_viral_df_valid = train_test_split(dem_viral_full_train, test_size = 0.2)

dem_nonviral_full_train, dem_nonviral_df_test = train_test_split(dem_nonviral_tweets_impt, test_size=0.2)
dem_nonviral_df_train, dem_nonviral_df_valid = train_test_split(dem_nonviral_full_train, test_size = 0.2)

rep_nonviral_full_train, rep_nonviral_df_test = train_test_split(rep_nonviral_tweets_impt, test_size=0.2)
rep_nonviral_df_train, rep_nonviral_df_valid = train_test_split(rep_nonviral_full_train, test_size = 0.2)

In [12]:
def build_dataset(df, dest_path):
    with open(dest_path, 'w') as f:  
      data = ''
      texts = df['Text'].tolist()
      for t in texts:
          bos_token = '<BOS>'
          eos_token = '<EOS>'
          data += bos_token + ' ' + t + ' ' + eos_token + '\n'
          
      f.write(data)

In [13]:
!rm -rf 'rep_viral'
!rm -rf 'rep_nonviral'
!rm -rf 'dem_viral'
!rm -rf 'dem_nonviral'

!mkdir 'rep_viral'
!mkdir 'rep_nonviral'
!mkdir 'dem_viral'
!mkdir 'dem_nonviral'

build_dataset(rep_viral_df_train, 'rep_viral/rep_viral_train.txt')
build_dataset(rep_viral_df_valid, 'rep_viral/rep_viral_valid.txt')
build_dataset(rep_viral_df_test, 'rep_viral/rep_viral_test.txt')

build_dataset(dem_viral_df_train, 'dem_viral/dem_viral_train.txt')
build_dataset(dem_viral_df_valid, 'dem_viral/dem_viral_valid.txt')
build_dataset(dem_viral_df_test,  'dem_viral/dem_viral_test.txt')

build_dataset(dem_nonviral_df_train, 'dem_nonviral/dem_nonviral_train.txt')
build_dataset(dem_nonviral_df_valid, 'dem_nonviral/dem_nonviral_valid.txt')
build_dataset(dem_nonviral_df_test,  'dem_nonviral/dem_nonviral_test.txt')

build_dataset(rep_nonviral_df_train, 'rep_nonviral/rep_nonviral_train.txt')
build_dataset(rep_nonviral_df_valid, 'rep_nonviral/rep_nonviral_valid.txt')
build_dataset(rep_nonviral_df_test,  'rep_nonviral/rep_nonviral_test.txt')

In [14]:
!ls '/content'

dem_nonviral  dem_viral  rep_nonviral  rep_viral  sample_data  Tweets.csv


In [15]:
import torch

cuda = torch.device('cuda')

In [16]:
# !ls '/content'

%env N=1
%env OUTPUT_DIR=/content/dem_viral_five_iter/
%env TRAIN_FILE=/content/dem_viral/dem_viral_train.txt
%env VALID_FILE=/content/dem_viral/dem_viral_valid.txt
%env CUDA_VISIBLE_DEVICES=0

!python run_clm.py \
--output_dir=$OUTPUT_DIR \
--model_type=gpt2 \
--model_name_or_path=gpt2 \
--dataset_name='dem_viral' \
--do_train \
--train_file=$TRAIN_FILE \
--do_eval \
--validation_file=$VALID_FILE \
--per_device_train_batch_size=2 \
--per_device_eval_batch_size=2 \
--learning_rate 3e-5 \
--num_train_epochs=20 \
--overwrite_output_dir

env: N=1
env: OUTPUT_DIR=/content/dem_viral_five_iter/
env: TRAIN_FILE=/content/dem_viral/dem_viral_train.txt
env: VALID_FILE=/content/dem_viral/dem_viral_valid.txt
env: CUDA_VISIBLE_DEVICES=0
python3: can't open file 'run_clm.py': [Errno 2] No such file or directory


In [17]:
# !ls '/content'

%env N=1
%env OUTPUT_DIR=/content/rep_viral_five_iter/
%env TRAIN_FILE=/content/rep_viral/rep_viral_train.txt
%env VALID_FILE=/content/rep_viral/rep_viral_valid.txt
%env CUDA_VISIBLE_DEVICES=0

!python run_clm.py \
--output_dir=$OUTPUT_DIR \
--model_type=gpt2 \
--model_name_or_path=gpt2 \
--dataset_name='rep_viral' \
--do_train \
--train_file=$TRAIN_FILE \
--do_eval \
--validation_file=$VALID_FILE \
--per_device_train_batch_size=2 \
--per_device_eval_batch_size=2 \
--learning_rate 7e-5 \
--num_train_epochs=10 \
--overwrite_output_dir

env: N=1
env: OUTPUT_DIR=/content/rep_viral_five_iter/
env: TRAIN_FILE=/content/rep_viral/rep_viral_train.txt
env: VALID_FILE=/content/rep_viral/rep_viral_valid.txt
env: CUDA_VISIBLE_DEVICES=0
python3: can't open file 'run_clm.py': [Errno 2] No such file or directory


In [18]:
# !ls '/content'

%env N=1
%env OUTPUT_DIR=/content/rep_nonviral_five_iter/
%env TRAIN_FILE=/content/rep_nonviral/rep_nonviral_train.txt
%env VALID_FILE=/content/rep_nonviral/rep_nonviral_valid.txt
%env CUDA_VISIBLE_DEVICES=0

!python run_clm.py \
--output_dir=$OUTPUT_DIR \
--model_type=gpt2 \
--model_name_or_path=gpt2 \
--dataset_name='rep_nonviral' \
--do_train \
--train_file=$TRAIN_FILE \
--do_eval \
--validation_file=$VALID_FILE \
--per_device_train_batch_size=2 \
--per_device_eval_batch_size=2 \
--learning_rate 5e-5 \
--num_train_epochs=30 \
--overwrite_output_dir

env: N=1
env: OUTPUT_DIR=/content/rep_nonviral_five_iter/
env: TRAIN_FILE=/content/rep_nonviral/rep_nonviral_train.txt
env: VALID_FILE=/content/rep_nonviral/rep_nonviral_valid.txt
env: CUDA_VISIBLE_DEVICES=0
python3: can't open file 'run_clm.py': [Errno 2] No such file or directory


In [19]:
# !ls '/content'

%env N=1
%env OUTPUT_DIR=/content/dem_nonviral_five_iter/
%env TRAIN_FILE=/content/dem_nonviral/dem_nonviral_train.txt
%env VALID_FILE=/content/dem_nonviral/dem_nonviral_valid.txt
%env CUDA_VISIBLE_DEVICES=0

!python run_clm.py \
--output_dir=$OUTPUT_DIR \
--model_type=gpt2 \
--model_name_or_path=gpt2 \
--dataset_name='dem_nonviral' \
--do_train \
--train_file=$TRAIN_FILE \
--do_eval \
--validation_file=$VALID_FILE \
--per_device_train_batch_size=2 \
--per_device_eval_batch_size=2 \
--learning_rate 5e-5 \
--num_train_epochs=17 \
--overwrite_output_dir

env: N=1
env: OUTPUT_DIR=/content/dem_nonviral_five_iter/
env: TRAIN_FILE=/content/dem_nonviral/dem_nonviral_train.txt
env: VALID_FILE=/content/dem_nonviral/dem_nonviral_valid.txt
env: CUDA_VISIBLE_DEVICES=0
python3: can't open file 'run_clm.py': [Errno 2] No such file or directory


In [20]:
# %env K=k_for_top-k_sampling_decoder
%env CUDA_VISIBLE_DEVICES=0
%env OUTPUT_DIR=/content/dem_viral_five_iter/

!python run_generation.py 2>&1 \
--model_type gpt2 \
--model_name_or_path $OUTPUT_DIR \
--length 300 \
--prompt "<BOS>" \
--stop_token "<EOS>" \
--num_return_sequences 100  | tee '/content/dem_viral.txt'

env: CUDA_VISIBLE_DEVICES=0
env: OUTPUT_DIR=/content/dem_viral_five_iter/
python3: can't open file 'run_generation.py': [Errno 2] No such file or directory


In [21]:
# %env K=k_for_top-k_sampling_decoder
%env CUDA_VISIBLE_DEVICES=0
%env OUTPUT_DIR=/content/rep_viral_five_iter/

!python run_generation.py 2>&1 \
--model_type gpt2 \
--model_name_or_path $OUTPUT_DIR \
--length 300 \
--prompt "<BOS>" \
--stop_token "<EOS>" \
--num_return_sequences 100  | tee '/content/rep_viral.txt'

env: CUDA_VISIBLE_DEVICES=0
env: OUTPUT_DIR=/content/rep_viral_five_iter/
python3: can't open file 'run_generation.py': [Errno 2] No such file or directory


In [22]:
# %env K=k_for_top-k_sampling_decoder
%env CUDA_VISIBLE_DEVICES=0
%env OUTPUT_DIR=/content/dem_nonviral_five_iter/

!python run_generation.py 2>&1 \
--model_type gpt2 \
--model_name_or_path $OUTPUT_DIR \
--length 300 \
--prompt "<BOS>" \
--stop_token "<EOS>" \
--num_return_sequences 100  | tee '/content/dem_nonviral.txt'

env: CUDA_VISIBLE_DEVICES=0
env: OUTPUT_DIR=/content/dem_nonviral_five_iter/
python3: can't open file 'run_generation.py': [Errno 2] No such file or directory


In [23]:
# %env K=k_for_top-k_sampling_decoder
%env CUDA_VISIBLE_DEVICES=0
%env OUTPUT_DIR=/content/rep_nonviral_five_iter/

!python run_generation.py 2>&1 \
--model_type gpt2 \
--model_name_or_path $OUTPUT_DIR \
--length 300 \
--prompt "<BOS>" \
--stop_token "<EOS>" \
--num_return_sequences 100  | tee '/content/rep_nonviral.txt'

env: CUDA_VISIBLE_DEVICES=0
env: OUTPUT_DIR=/content/rep_nonviral_five_iter/
python3: can't open file 'run_generation.py': [Errno 2] No such file or directory


In [ ]:
!zip -r /content/rep_model.zip /content/my_model_five_iter

  adding: content/my_model_five_iter/ (stored 0%)
  adding: content/my_model_five_iter/all_results.json (deflated 55%)
  adding: content/my_model_five_iter/tokenizer.json (deflated 72%)
  adding: content/my_model_five_iter/vocab.json (deflated 59%)
  adding: content/my_model_five_iter/training_args.bin (deflated 49%)
  adding: content/my_model_five_iter/special_tokens_map.json (deflated 38%)
  adding: content/my_model_five_iter/README.md (deflated 50%)
  adding: content/my_model_five_iter/pytorch_model.bin (deflated 9%)
  adding: content/my_model_five_iter/trainer_state.json (deflated 53%)
  adding: content/my_model_five_iter/runs/ (stored 0%)
  adding: content/my_model_five_iter/runs/Apr17_18-30-45_f4c91529bdc1/ (stored 0%)
  adding: content/my_model_five_iter/runs/Apr17_18-30-45_f4c91529bdc1/events.out.tfevents.1650220253.f4c91529bdc1.389.0 (deflated 57%)
  adding: content/my_model_five_iter/runs/Apr17_18-30-45_f4c91529bdc1/events.out.tfevents.1650220625.f4c91529bdc1.389.2 (deflated 

In [ ]:
!zip -r /content/dem_viral.zip /content/dem_viral_five_iter
!zip -r /content/dem_nonviral.zip /content/dem_nonviral_five_iter
!zip -r /content/rep_viral.zip /content/rep_viral_five_iter
!zip -r /content/rep_nonviral.zip /content/rep_nonviral_five_iter

  adding: content/dem_viral_five_iter/ (stored 0%)
  adding: content/dem_viral_five_iter/all_results.json (deflated 54%)
  adding: content/dem_viral_five_iter/merges.txt (deflated 53%)
  adding: content/dem_viral_five_iter/eval_results.json (deflated 42%)
  adding: content/dem_viral_five_iter/config.json (deflated 51%)
  adding: content/dem_viral_five_iter/train_results.json (deflated 40%)
  adding: content/dem_viral_five_iter/pytorch_model.bin (deflated 9%)
  adding: content/dem_viral_five_iter/special_tokens_map.json (deflated 38%)
  adding: content/dem_viral_five_iter/added_tokens.json (deflated 31%)
  adding: content/dem_viral_five_iter/tokenizer_config.json (deflated 40%)
  adding: content/dem_viral_five_iter/training_args.bin (deflated 48%)
  adding: content/dem_viral_five_iter/tokenizer.json (deflated 72%)
  adding: content/dem_viral_five_iter/vocab.json (deflated 59%)
  adding: content/dem_viral_five_iter/trainer_state.json (deflated 53%)
  adding: content/dem_viral_five_iter/r

In [ ]:
from google.colab import files

files.download('/content/rep_nonviral.zip')
files.download('/content/dem_nonviral.zip')
files.download('/content/dem_viral.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>